In [1]:
import sys
import os
import pandas as pd
sys.path.append('/afs/cern.ch/cms/PPD/PdmV/tools/McM/')
from rest import McM

### McM scripting [tutorial](https://indico.cern.ch/event/894961/contributions/3774016/attachments/2005333/3349474/McM_Scripting_2020_03_18.pdf)
### Documentation: [restapi](https://cms-pdmv.cern.ch/mcm/restapi)

In [2]:
mcm = McM(dev=False)
tags = '20230328_mkomm_HeavyNeutrino_dilepton'
results = mcm.get('requests', query=f'tags={tags}&status=new&approval=none')
len(results)

[2023-04-26 17:03:48,678][INFO] Using SSO cookie file /afs/cern.ch/user/d/dfu/private/mcm-prod-cookie.txt
[2023-04-26 17:03:48,915][WARNING] Most likely SSO cookie is expired, will remake it after 1 seconds
[2023-04-26 17:03:51,093][INFO] Using SSO cookie file /afs/cern.ch/user/d/dfu/private/mcm-prod-cookie.txt


6

In [3]:
CLEAR=False
if CLEAR:
    for r in results:
        mcm.delete(object_type='requests', object_id=r['_id'])

In [3]:
results[0]['fragment']

'import FWCore.ParameterSet.Config as cms\n\nexternalLHEProducer = cms.EDProducer("ExternalLHEProducer",\n    args = cms.vstring(\'/cvmfs/cms.cern.ch/phys_generator/gridpacks/pre2017/13TeV/madgraph/V5_2.6.5/HNL_majorana_all/HNL_majorana_all_ctau1p0e00_massHNL10p0_Vall1p177e-03_tarball.tar.xz\'),\n    nEvents = cms.untracked.uint32(5000),\n    numberOfParameters = cms.uint32(1),\n    outputFile = cms.string(\'cmsgrid_final.lhe\'),\n    # generately = cms.untracked.bool(True),\n    scriptName = cms.FileInPath(\'GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh\')\n)\n\n# Link to cards:\n# N/A\n\nimport FWCore.ParameterSet.Config as cms\n\nfrom Configuration.Generator.Pythia8CommonSettings_cfi import *\nfrom Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *\n\ngenerator = cms.EDFilter("Pythia8HadronizerFilter",\n    maxEventsToPrint = cms.untracked.int32(1),\n    pythiaPylistVerbosity = cms.untracked.int32(1),\n    filterEfficiency = cms.untracked.double(1

In [6]:
results[2]['fragment']

'import FWCore.ParameterSet.Config as cms\n\nexternalLHEProducer = cms.EDProducer("ExternalLHEProducer",\n    args = cms.vstring(\'/cvmfs/cms.cern.ch/phys_generator/gridpacks/pre2017/13TeV/madgraph/V5_2.6.5/HNL_majorana_all/HNL_majorana_all_ctau1p0e00_massHNL10p0_Vall1p177e-03_tarball.tar.xz\'),\n    nEvents = cms.untracked.uint32(5000),\n    numberOfParameters = cms.uint32(1),\n    outputFile = cms.string(\'cmsgrid_final.lhe\'),\n    generateConcurrently = cms.untracked.bool(False),\n    scriptName = cms.FileInPath(\'GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh\')\n)\n\n# Link to cards:\n# N/A\n\nimport FWCore.ParameterSet.Config as cms\n\nfrom Configuration.Generator.Pythia8CommonSettings_cfi import *\nfrom Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *\n\ngenerator = cms.EDFilter("Pythia8HadronizerFilter",\n    maxEventsToPrint = cms.untracked.int32(1),\n    pythiaPylistVerbosity = cms.untracked.int32(1),\n    filterEfficiency = cms.untracked

In [3]:
for r in results:
    #r['fragment'] = r['fragment'].replace('generately', '# generately')
    r['time_event'] = [9]
    r['size_event'] = [535]
    #r['mcdb_id'] = 0
    #print(r['dataset_name'].split('_M')[1].split('_')[0], r['fragment'].split("GRIDPACK = \'")[1].split("\' %")[0])

In [ ]:
for r in results:
    output = mcm.update('requests', r)
    if output['results'] == False:
        print(r['_id'], output['message'])

In [8]:
time = set()
for r in results:
    time.add(r['time_event'][0])
time

{50}